# amplitude trends

In [1]:
include("/Net/Groups/BGI/people/gkraemer/progs/data_cube/julia/notebooks/stuff.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-14254504493278934301\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-6145919900333710188\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/user/gkraemer//assetserver/549eec07f928a744cd5cefcd9652eb9af414188d-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-14254504493278934301\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

┌ Warning: Package RCall does not have AxisArrays in its dependencies:
│ - If you have RCall checked out for development and have
│   added AxisArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with RCall
│ Loading AxisArrays into RCall from project dependency, future warnings for RCall are suppressed.
└ @ nothing nothing:840
┌ Warning: RCall.jl: Loading required package: sp
└ @ RCall /Net/Groups/BGI/people/gkraemer/.julia_atacama/packages/RCall/29zDq/src/io.jl:113
┌ Warning: RCall.jl: Checking rgeos availability: TRUE
└ @ RCall /Net/Groups/BGI/people/gkraemer/.julia_atacama/packages/RCall/29zDq/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: zoo
│ 
│ Attaching package: 'zoo'
│ 
│ The following objects are masked from 'package:base':
│ 
│     as.Date, as.Date.numeric
│ 
│ Loading required package: sandwich
└ @ RCall /Net/Groups/BGI/people/gkraemer/.julia_atacama/packag

PcaAxis             Axis with 12 elements: PCA_1 PCA_2 .. PCA_11 PCA_12 

In [2]:
pca_y_amplitude_trends = loadCube(joinpath(cubes_base_dir, "pca_y_amplitude_trends"))

Memory mapped cube with the following dimensions
PcaAxis             Axis with 12 elements: PCA_1 PCA_2 .. PCA_11 PCA_12 
Lon                 Axis with 1440 Elements from -179.875 to 179.875
Lat                 Axis with 720 Elements from 89.875 to -89.875
Total size: 59.33 MB


In [22]:
R"""
#options(device = function(filename=getOption('rcalljl_filename'),...) png(filename, width = 700, height = 500, ...))
data(coastsCoarse, package = "rworldmap")

from_to <- 120:670
elem_col = "black" # "gray50"

pal1 <- rev(c("#543005", "#8C510A", "#BF812D", "#DFC27D", 
              "#F6E8C3", "#F5F5F5", "#C7EAE5", "#80CDC1", 
              "#35978F", "#01665E", "#003C30"))
pal1 <- colorRampPalette(colors = pal1)(100)

pal2 <- rev(c("#67001f", "#b2182b","#d6604d", "#f4a582",
              "#fddbc7", "#f7f7f7", "#d1e5f0","#92c5de",
              "#4393c3", "#2166ac","#053061"))
pal2 <- colorRampPalette(colors = pal2)(100)

pal3 <- c("#7f3b08","#b35806","#e08214","#fdb863",
          "#fee0b6","#f7f7f7","#d8daeb","#b2abd2",
          "#8073ac","#542788","#2d004b")
pal3 <- colorRampPalette(colors = pal3)(100)

data_pc1 <- $(pca_y_amplitude_trends[1, :, :] |> collect |> x -> x[:, end:-1:1] |> missing_to_nan .|> Float64)
data_pc2 <- $(pca_y_amplitude_trends[2, :, :] |> collect |> x -> x[:, end:-1:1] |> missing_to_nan .|> Float64)
data_pc3 <- $(pca_y_amplitude_trends[3, :, :] |> collect |> x -> x[:, end:-1:1] |> missing_to_nan .|> Float64)

ex1 <- range(data_pc1, na.rm = TRUE)
ex1 %<>% abs %>% max %>% {c(-., .) * 0.9}

ex2 <- range(data_pc2, na.rm = TRUE)
ex2 %<>% abs %>% max %>% {c(-., .) * 0.9}

ex3 <- range(data_pc3, na.rm = TRUE)
ex3 %<>% abs %>% max %>% {c(-., .) * 0.9}


pdf($(joinpath(fig_path, "amp_slope_pc123.pdf")), width = 7, height = 7 * length(from_to) / 720 * 3/2)

layout(t(t(1:3)), heights = c(1, 1, 1))
par(mar = c(0,0,0,0))

image(
    seq(-180, 180, by = 0.25),
    seq(-60, 77.5, by = 0.25),
    data_pc1[,from_to], 
    zlim = ex1, 
    col = pal1,
    bty = "n",
    axes = FALSE,
    useRaster = TRUE    
)
plot(coastsCoarse, add = TRUE)

image(
    seq(-180, 180, by = 0.25),
    seq(-60, 77.5, by = 0.25),
    data_pc2[,from_to], 
    zlim = ex2, 
    col = pal2,
    bty = "n",
    axes = FALSE,
    useRaster = TRUE
)
plot(coastsCoarse, add = TRUE)

image(
    seq(-180, 180, by = 0.25),
    seq(-60, 77.5, by = 0.25),
    data_pc3[,from_to], 
    zlim = ex3, 
    col = pal3,
    bty = "n",
    axes = FALSE,
    useRaster = TRUE
)
plot(coastsCoarse, add = TRUE)

par(fig = c(0.1, 0.12, (2 + 0.01) / 3, (2 + 0.7) / 3), new = TRUE, xpd = NA)
image(
    0:1,
    seq(from = ex1[1], to = ex1[2], length.out = 100), 
    t(seq(from = ex1[1], to = ex1[2], length.out = 100)), 
    zlim = ex1, col = pal1,
    axes = FALSE, xlab = NA,
    col.lab = elem_col,
    useRaster = TRUE,
    ylab = expression(Slope~group("[", std ~ y^-1, "]"))
)
#box(col = elem_col)
axis(2, las = 1, col = elem_col, col.axis = elem_col)

par(fig = c(0.1, 0.12, (1 + 0.01) / 3, (1 + 0.7) / 3), new = TRUE)
image(
    0:1,
    seq(from = ex2[1], to = ex2[2], length.out = 100), 
    t(seq(from = ex2[1], to = ex2[2], length.out = 100)), 
    zlim = ex2, col = pal2,
    axes = FALSE, xlab = NA,
    col.lab = elem_col,
    useRaster = TRUE,
    ylab = expression(Slope~group("[", std ~ y^-1, "]"))
)
#box(col = elem_col)
axis(2, las = 1, col = elem_col, col.axis = elem_col, xpd = NA)

par(fig = c(0.1, 0.12, 0.01 / 3, 0.7 / 3), new = TRUE)
image(
    0:1,
    seq(from = ex3[1], to = ex3[2], length.out = 100), 
    t(seq(from = ex3[1], to = ex3[2], length.out = 100)), 
    zlim = ex3, col = pal3,
    axes = FALSE, xlab = NA,
    col.lab = elem_col,
    useRaster = TRUE,
    ylab = expression(Slope~group("[", std ~ y^-1, "]"))
)
#box(col = elem_col)
axis(2, las = 1, col = elem_col, col.axis = elem_col, xpd = NA)


grid.text("(a)", 0, 1,   just = c(0, 1))
grid.text("(b)", 0, 2/3, just = c(0, 1))
grid.text("(c)", 0, 1/3, just = c(0, 1))

dev.off()
"""

show_pdf(
    "https://localhost:8000/user/gkraemer/files/scratch/gkraemer/global_pca_j1/fig/amp_slope_pc123.pdf",
    width = 700,
    height = 1000
)

(object { attributes=Dict(:type=>"application/pdf",:height=>"1000",:data=>"https://localhost:8000/user/gkraemer/files/scratch/gkraemer/global_pca_j1/fig/amp_slope_pc123.pdf",:width=>"700") }
  "pdf not found")